# Download data from OpenSky API

Contrary to the osstate_vector database, this one will download data directly from the Trino database.

In [17]:
import traffic 
import os
import sys
# Get the current notebook's absolute directory
notebook_dir = os.path.abspath('')
# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)

from path_prefix import PATH_PREFIX

In [1]:
# Columns: time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,serials,hour
# Table name: state_vectors_data4

Some cool working SQL commands:

SELECT DISTINCT callsign, rand() AS random_value 
FROM state_vectors_data4 TABLESAMPLE BERNOULLI(1) 
WHERE lat BETWEEN 35 AND 72  
    AND lon BETWEEN -10 AND 40  
    AND hour = 1480762800
ORDER BY random_value
LIMIT 2000;

./trino.jar --user=thinhhoangdinh --password --server=https://trino.opensky-network.org --external-authentication --catalog "minio" --schema "osky" --execute="SELECT DISTINCT callsign, rand() AS random_value 
> FROM state_vectors_data4 TABLESAMPLE BERNOULLI(1) 
> WHERE lat BETWEEN 35 AND 72  
>     AND lon BETWEEN -10 AND 40  
>     AND hour = 1716520680
>     AND time-lastcontact <= 15
> ORDER BY random_value
> LIMIT 2000;" --output-format CSV > output.csv

SELECT * FROM state_vectors_data4 v JOIN (SELECT QUOTIENT(time, 60) AS minute, MAX(time) AS recent, icao24 FROM state_vectors_data4 WHERE hour=1480762800 GROUP BY icao24, minute) AS m ON v.icao24=m.icao24 AND v.time=m.recent WHERE v.hour=1480762800;

SELECT v.*  -- Select all columns from the main table (v)
FROM state_vectors_data4 v 
JOIN (
    SELECT 
        QUOTIENT(time, 60) AS minute, 
        MAX(time) AS recent_time 
    FROM state_vectors_data4 
    WHERE hour = 1716520680  -- Use your original hour value
    GROUP BY minute
) AS m ON QUOTIENT(v.time, 60) = m.minute AND v.time = m.recent_time
WHERE 
    v.lat BETWEEN 35 AND 72  
    AND v.lon BETWEEN -10 AND 40  
    AND v.hour = 1716520680
    AND v.time - v.lastcontact <= 15
LIMIT 50000;

SELECT v.*  -- Select all columns from the main table (v)
FROM state_vectors_data4 v 
WHERE 
    v.lat BETWEEN 35 AND 72  
    AND v.lon BETWEEN -10 AND 40  
    AND v.hour = 1716520680
    AND v.time - v.lastcontact <= 15
LIMIT 2000; 



To create our dataset, we will: 
- sample 2000 callsigns for each hour, obtain the callsign list


In [14]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# Define the date and time in Hanoi time
hanoi_time = datetime(2024, 5, 24, 10, 0, 0, tzinfo=ZoneInfo("Asia/Ho_Chi_Minh"))

# Convert to UTC
utc_time = hanoi_time.astimezone(timezone.utc)

# Convert to Unix timestamp
unix_timestamp = int(utc_time.timestamp())

print(f"10AM May 24, 2024 (Hanoi time) in Unix timestamp: {unix_timestamp}")


10AM May 24, 2024 (Hanoi time) in Unix timestamp: 1716519600


In [5]:
from datetime import datetime, timezone

unix_timestamp = 1590364810

# Convert Unix timestamp to datetime object in UTC
utc_datetime = datetime.fromtimestamp(unix_timestamp, tz=timezone.utc)

# Format the datetime object as a string
formatted_datetime = utc_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

print(f"The Unix timestamp {unix_timestamp} corresponds to: {formatted_datetime}")


The Unix timestamp 1590364810 corresponds to: 2020-05-25 00:00:10 UTC


v.time, v.icao24, v.lat, v.lon, v.heading, v.callsign, v.geoaltitude

SELECT v.time, v.icao24, v.lat, v.lon, v.heading, v.callsign, v.geoaltitude  FROM state_vectors_data4 v WHERE v.lat BETWEEN 35 AND 72 AND v.lon BETWEEN -10 AND 40 AND v.hour = 1716519600 AND v.time - v.lastcontact <= 15 LIMIT 20000;

SELECT 
    v.time, v.icao24, v.lat, v.lon, v.heading, v.callsign, v.geoaltitude  -- Select all columns from the main table (state_vectors_data4)

FROM 
    state_vectors_data4 v  -- Main table alias 'v'

JOIN (
    SELECT 
        FLOOR(time / 60) AS minute,  -- Calculate the minute from the timestamp
        MAX(time) AS recent_time     -- Get the most recent timestamp within each minute
    FROM 
        state_vectors_data4
    WHERE 
        hour = 1716519600  -- Filter by the specific hour
    GROUP BY 
        FLOOR(time / 60)  -- Group by minute
) AS m  -- Subquery alias 'm'

ON 
    FLOOR(v.time / 60) = m.minute  -- Join condition: Match minutes between main table and subquery
    AND v.time = m.recent_time     -- Join condition: Select only the most recent record within each minute

WHERE 
    v.lat BETWEEN 35 AND 72        -- Filter by latitude range
    AND v.lon BETWEEN -10 AND 40     -- Filter by longitude range
    AND v.hour = 1716519600         -- Filter by the specific hour
    AND v.time - v.lastcontact <= 15  -- Filter based on time difference

LIMIT 2000;  -- Limit the result to 2000 rows

./trino.jar --user=thinhhoangdinh --password --server=https://trino.opensky-network.org --external-authentication --catalog "minio" --schema "osky" --execute="SELECT \
    v.time, v.icao24, v.lat, v.lon, v.heading, v.callsign, v.geoaltitude \
FROM \
    state_vectors_data4 v \
JOIN ( \
    SELECT \
        FLOOR(time / 60) AS minute, \
        MAX(time) AS recent_time \
    FROM \
        state_vectors_data4 \
    WHERE \
        hour = 1716519600 \
    GROUP BY \
        FLOOR(time / 60) \
) AS m \
ON \
    FLOOR(v.time / 60) = m.minute \
    AND v.time = m.recent_time \
WHERE \
    v.lat BETWEEN 35 AND 72 \
    AND v.lon BETWEEN -10 AND 40 \
    AND v.hour = 1716519600 \
    AND v.time - v.lastcontact <= 15;" --output-format CSV > 1716519600.csv 

# Shell Commands Automation

In [19]:
import subprocess
import datetime
import pandas as pd

def execute_trino_commands(from_datetime, to_datetime):
    """
    Executes Trino shell commands for each hour within the specified datetime range.

    Args:
        from_datetime (datetime): The starting datetime.
        to_datetime (datetime): The ending datetime.
    """

    # Generate a list of Unix timestamps representing the beginning of each hour 
    hourly_timestamps = pd.date_range(from_datetime, to_datetime, freq='H').astype(int) // 10**9

    for timestamp in hourly_timestamps:
        print("Current timestamp: ", timestamp)
        # Construct the shell command, incorporating the timestamp into the relevant parts
        command = f"./trino.jar --user=thinhhoangdinh --password --server=https://trino.opensky-network.org --external-authentication --catalog 'minio' --schema 'osky' --execute='SELECT \
            v.time, v.icao24, v.lat, v.lon, v.heading, v.callsign, v.geoaltitude \
        FROM \
            state_vectors_data4 v \
        JOIN ( \
            SELECT \
                FLOOR(time / 60) AS minute, \
                MAX(time) AS recent_time \
            FROM \
                state_vectors_data4 \
            WHERE \
                hour = {timestamp} \
            GROUP BY \
                FLOOR(time / 60) \
        ) AS m \
        ON \
            FLOOR(v.time / 60) = m.minute \
            AND v.time = m.recent_time \
        WHERE \
            v.lat BETWEEN 35 AND 72 \
            AND v.lon BETWEEN -10 AND 40 \
            AND v.hour = {timestamp} \
            AND v.time - v.lastcontact <= 15;' --output-format CSV > {timestamp}.csv" 

        # Execute the shell command
        subprocess.run(command, shell=True)  # Use shell=True for proper command parsing


In [20]:
# Example usage
from_datetime = datetime.datetime(2024, 5, 24, 3, 0, 0)  # Adjust as needed
to_datetime = datetime.datetime(2024, 5, 24, 4, 0, 0)    # Adjust as needed

execute_trino_commands(from_datetime, to_datetime)

/var/folders/lm/mmss816142xg1l8dk_cq_j_h0000gn/T/ipykernel_2828/4063359239.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_timestamps = pd.date_range(from_datetime, to_datetime, freq='H').astype(int) // 10**9


Current timestamp:  1716519600


Sep 03, 2024 9:20:55 AM org.jline.utils.Log logr


KeyboardInterrupt: 

In [22]:
import requests 

result = requests.post(
    "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token",
    data={
        "client_id": "trino-client",
        "grant_type": "password",
        "username": "thinhhoangdinh",
        "password": "iQ6^yrwe7o3m",
    }
)

print(result.json())

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ0SVIwSDB0bmNEZTlKYmp4dFctWEtqZ0RYSWExNnR5eU5DWHJxUzJQNkRjIn0.eyJleHAiOjE3MjUzNTY4NjUsImlhdCI6MTcyNTM0OTY2NSwianRpIjoiMGNiYzYxZTEtNDI2Mi00MDA3LTg5MTQtZTgxN2EzNjRmM2M5IiwiaXNzIjoiaHR0cHM6Ly9hdXRoLm9wZW5za3ktbmV0d29yay5vcmcvYXV0aC9yZWFsbXMvb3BlbnNreS1uZXR3b3JrIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjEzYmYwYmQwLTMzOTktNDA2NS04ZGFiLTIyYzI0Njg1N2E4MSIsInR5cCI6IkJlYXJlciIsImF6cCI6InRyaW5vLWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIxZjQ2MzhhMy0wYjk4LTRmYzctOWNlOC1jZDBiOWJiMGI1N2UiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtb3BlbnNreS1uZXR3b3JrIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6IjFmNDYzOGEzLTBiOTgtNGZjNy05Y2U4LWNkMGI5YmIwYjU3ZSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiVGhpbmggSG9hbmciLCJncm91cHMiOlsiL29wZW5za3kvdHJpbm8vcmVhZG9ubHkiXSwicH